# 4D Inversion on Field Data
This notebook requires a trained model from the notebook in [`01 - 4D-Pressure-Saturation-Inversion`](01 - 4D-Pressure-Saturation-Inversion.ipynb).

In [ ]:
import os

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np

from scipy.io import loadmat, savemat

from sklearn.model_selection import train_test_split
%load_ext autoreload
%autoreload 2

In [ ]:
from numpy.random import seed
seed(42)
from tensorflow import set_random_seed
set_random_seed(42)

# Data
This notebook evaluates the network on field data.

In [ ]:
def data_gen(timestep):
    seed(42)
    set_random_seed(42)
    #suffixes = [""]
    location = "data"
    filename = "Seis2PS_NN_training_input"
    pore_volume = loadmat(os.path.join(location,filename))["Pore_volume"]
    location = "data"
    filename = "Seis2PS_NN_training_input_obs"
    seis_ps = loadmat(os.path.join(location,filename))
    pore_volume
    headers = ["dSNA_nr", "dSNA_md", "dSNA_fr"]
    out_test = pd.DataFrame()
    tost = pd.DataFrame()
    for x in headers:
        ravel_data = seis_ps[x][0][timestep]
        out_test[x] = ravel_data.ravel()
    
    out_test["Pore_volume"] = pore_volume.ravel()
    
    
    
    return out_test
data_gen(1).describe()

In [ ]:
import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, AlphaDropout, Dropout
from keras import regularizers
from keras import optimizers 
from keras import callbacks

from keras_tqdm import TQDMNotebookCallback 

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, quniform, uniform, loguniform

from sklearn.metrics import r2_score
%autoreload 2

In [ ]:
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

def r_square_loss(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return 1 - ( 1 - SS_res/(SS_tot + K.epsilon()))

def huber_loss(y_true, y_pred, clip_delta=.35):
    error = y_true - y_pred
    cond  = tf.keras.backend.abs(error) < clip_delta
    squared_loss = 0.5 * tf.keras.backend.square(error)
    linear_loss  = clip_delta * (tf.keras.backend.abs(error) - 0.5 * clip_delta)
    
    return tf.where(cond, squared_loss, linear_loss)

In [ ]:
experiment = "publication"

## Model Loading
We load the noise-trained model. We have to provide the custom_objects `r_square`, `huber_loss` for the experiments.

In [ ]:
model = keras.models.load_model("best.hd5", custom_objects={'r_square': r_square, 'huber_loss': huber_loss})

In [ ]:
model.summary()

In [ ]:
q = 3
X_test = data_gen(q)

In [ ]:
preddata = model.predict(X_test)

## Plotting
Plot the resulting inversion and the networks.

In [ ]:
for q in range(3,4):
    X_test = data_gen(q)
    preddata = model.predict([X_test["dSNA_nr"],X_test["dSNA_md"],X_test["dSNA_fr"],X_test["Pore_volume"]])
    fig = plt.figure(figsize=(15, 10))
    plt.subplot(231)
    vmax=np.max(np.abs(seis_ps["dSNA_nr"][0][q]))
    plt.imshow(seis_ps["dSNA_nr"][0][q], cmap="seismic_r", vmin=-vmax, vmax=vmax,  aspect='equal')
    plt.colorbar()
    plt.title("Near")
    plt.subplot(232)
    vmax=np.max(np.abs(seis_ps["dSNA_md"][0][q]))
    plt.imshow(seis_ps["dSNA_md"][0][q], cmap="seismic_r", vmin=-vmax, vmax=vmax,  aspect='equal')
    plt.colorbar()
    plt.title("Mid")
    plt.subplot(233)
    vmax=np.max(np.abs(seis_ps["dSNA_fr"][0][q]))
    plt.imshow(seis_ps["dSNA_fr"][0][q], cmap="seismic_r", vmin=-vmax, vmax=vmax,  aspect='equal')
    plt.colorbar()
    plt.title("Far")
    
    ax0 = plt.subplot(234)
    data = preddata[0].reshape(seis_ps["dSNA_md"][0][0].shape)
    vmax = np.nanmax(np.abs(data))
    im0 = ax0.imshow(data, cmap="seismic", vmin=-vmax, vmax=vmax, aspect='equal')
    ax0.set_title("Neural Network dP "+experiment)
    fig.colorbar(im0, ax=ax0)
    
    ax1 = plt.subplot(235)
    data = preddata[1].reshape(seis_ps["dSNA_md"][0][0].shape)
    vmax = np.nanmax(np.abs(data))
    im1 = ax1.imshow(data, cmap="seismic_r", vmin=-vmax, vmax=vmax, aspect='equal')
    ax1.set_title("Neural Network dSw")
    fig.colorbar(im1, ax=ax1)
    
    
    ax2 = plt.subplot(236)
    data = preddata[2].reshape(seis_ps["dSNA_md"][0][0].shape)
    vmax = np.nanmax(np.abs(data))
    im2 = ax2.imshow(data, cmap="seismic", vmin=-vmax, vmax=vmax, aspect='equal')
    ax2.set_title("Neural Network dSg")
    fig.colorbar(im2, ax=ax2)
    
    blerg = "obs"
    mat_dict = {blerg+"dP_nn_data": preddata[0].reshape(seis_ps["dSNA_md"][0][0].shape),
                blerg+"dSw_nn_data": preddata[1].reshape(seis_ps["dSNA_md"][0][0].shape),
                blerg+"dSg_nn_data": preddata[2].reshape(seis_ps["dSNA_md"][0][0].shape),
                blerg+"near": seis_ps["dSNA_nr"][0][q],
                blerg+"mid": seis_ps["dSNA_md"][0][q],
                blerg+"far": seis_ps["dSNA_fr"][0][q],}
    savemat('matlab/'+blerg+'.mat', mat_dict)
    
    fig.savefig("Observed-vae-gradient-basemod2-timestep"+str(q)+".png")
    fig.show()

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
X_test.shape

In [ ]:
preddata[0].shape

In [ ]:
location = "data"
filename = "Seis2PS_NN_training_input"
pore_volume = loadmat(os.path.join(location,filename))["Pore_volume"]

In [ ]:
location = "data"
filename = "Seis2PS_NN_training_input_obs"
seis_ps = loadmat(os.path.join(location,filename))
headers = list(seis_ps.keys())[3:]
plt.imshow(pore_volume)

In [ ]:
seis_ps["dSNA_nr"][0][0].shape

In [ ]:
plt.figure(figsize=(15,20))
plt.subplot(2, 2, 1)
vmax=np.max(np.abs(seis_ps["dSNA_nr"][0][q]))
plt.imshow(seis_ps["dSNA_nr"][0][q], cmap="seismic_r", vmin=-vmax, vmax=vmax,  aspect='auto')
plt.colorbar()
plt.title("Near")
plt.subplot(2, 2, 2)
vmax=np.max(np.abs(seis_ps["dSNA_md"][0][q]))
plt.imshow(seis_ps["dSNA_md"][0][q], cmap="seismic_r", vmin=-vmax, vmax=vmax,  aspect='auto')
plt.colorbar()
plt.title("Mid")
plt.subplot(2, 2, 3)
vmax=np.max(np.abs(seis_ps["dSNA_fr"][0][q]))
plt.imshow(seis_ps["dSNA_fr"][0][q], cmap="seismic_r", vmin=-vmax, vmax=vmax,  aspect='auto')
plt.colorbar()
plt.title("Far")
plt.subplot(2, 2, 4)
vmax=np.nanmax(seis_ps["Pore_volume"])
plt.imshow(seis_ps["Pore_volume"], cmap="seismic_r", vmin=-vmax, vmax=vmax,  aspect='auto')
plt.colorbar()
plt.title("Pore Volume")
plt.savefig("Seismic-input.png")
plt.show()